In [1]:
# !pip install -r requirements.txt

In [2]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(900_000_000_000) ## 800GB?

In [3]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

from utilities import *
import shared_utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [4]:
import sys

In [5]:
sys.path.append('../rt_delay/')

In [6]:
import utils as rt_utils

[PRC 21064.3](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=21064.3.&lawCode=PRC)
* _Major transit stop means a site containing any of the following:
(a) An existing rail or bus rapid transit station.
(b) A ferry terminal served by either a bus or rail transit service.
(c) The intersection of two or more major bus routes with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods._
    * "Intersection" may not be sufficiently well-defined for this analysis

[PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit)
* _(a) “Bus rapid transit” means a public mass transit service provided by a public agency or by a public-private partnership that includes all of the following features:
(1) Full-time dedicated bus lanes or operation in a separate right-of-way dedicated for public transportation with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods.
(2) Transit signal priority.
(3) All-door boarding.
(4) Fare collection system that promotes efficiency.
(5) Defined stations._
    * Unlikely to determine if a service qualifies as BRT under this definition using GTFS alone

### Rail Service

* temporary workaround with dim tables, should use tbl.gtfs schedule but LA Metro rail not in

In [7]:
def get_routes_by_type(route_types, analysis_date):
    routes_on_date = (tbl.views.gtfs_schedule_fact_daily_feed_routes()
         >> filter(_.date == analysis_date)
         >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
        )

    dim_routes = tbl.views.gtfs_schedule_dim_routes()
    routes_date_joined = (routes_on_date
         >> inner_join(_, dim_routes >> select(_.route_id, _.route_key, _.route_short_name,
                                                   _.route_long_name, _.route_desc, _.route_type,
                                              _.calitp_itp_id),
                       on = 'route_key')
         # >> distinct(_.calitp_itp_id, _.route_id, _.route_short_name, _.route_long_name, _.route_desc, _.route_type)
         >> filter(_.calitp_itp_id != 200) # avoid MTC feed in favor of individual operator feeds
         >> filter((_.route_type.isin(route_types)))
         # >> collect()
        )
    return routes_date_joined

In [8]:
analysis_date = dt.date(2022, 6, 15) ## Wed, June 15

In [9]:
rail_route_types = ['0', '1', '2']
new_routes = get_routes_by_type(rail_route_types, analysis_date)

In [10]:
ca = gpd.read_parquet(f'{GCS_FILE_PATH}ca_boundary.parquet')

In [11]:
def routes_to_stops(routes_tbl, analysis_date):
    
    trips_query = (tbl.views.gtfs_schedule_fact_daily_trips()
    >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
    >> filter(_.service_date == analysis_date)
    >> filter(_.is_in_service == True)
    >> select(_.trip_key, _.service_date, _.route_id, _.calitp_itp_id)
    >> inner_join(_, routes_tbl, on = ['calitp_itp_id', 'route_id'])
    )
    trips_ix_query = (trips_query
    >> inner_join(_, tbl.views.gtfs_schedule_index_feed_trip_stops(), on = 'trip_key')
    >> select(-_.calitp_url_number, -_.calitp_extracted_at, -_.calitp_deleted_at)
    )
    stops = (tbl.views.gtfs_schedule_dim_stops()
     >> distinct(_.calitp_itp_id, _.stop_id,
              _.stop_lat, _.stop_lon, _.stop_name, _.stop_key)
     >> inner_join(_, trips_ix_query >> distinct(_.stop_key, _.route_type), on = 'stop_key')
     >> collect()
     >> distinct(_.calitp_itp_id, _.stop_id, _keep_all=True) ## should be ok to drop duplicates, but must use stop_id for future joins...
     >> select(-_.stop_key)
    )

    stops = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
                            crs='EPSG:4326').to_crs(shared_utils.geography_utils.CA_NAD83Albers)
    
    return stops.clip(ca)

In [12]:
rail_stops = routes_to_stops(new_routes, analysis_date)

In [14]:
# map_hqta(rail_stops, 'route_type')

#### BRT Service likely meeting [PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit) definition:

* LA Metro Orange, Silver excluding street running (stop flags only)
* ~~Omnitrans sbX, all stops (curbside stations are well-defined, with fare prepayment)~~
    * insufficient frequency 5/16
* AC Transit Tempo, all stops (curbside stations are well-defined, with fare prepayment)

[PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit)
* _(a) “Bus rapid transit” means a public mass transit service provided by a public agency or by a public-private partnership that includes all of the following features:
(1) Full-time dedicated bus lanes or operation in a separate right-of-way dedicated for public transportation with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods.
(2) Transit signal priority.
(3) All-door boarding.
(4) Fare collection system that promotes efficiency.
(5) Defined stations._
    * Unlikely to determine if a service qualifies as BRT under this definition using GTFS alone

In [15]:
metro_routes = rt_utils.get_routes(182, analysis_date)

In [16]:
metro_brt = metro_routes >> filter(_.route_id.str.contains('901') |
                      _.route_id.str.contains('910'))

In [17]:
metro_brt_stops = routes_to_stops(metro_brt, analysis_date)

In [18]:
## unable to filter out non-station stops using GTFS, manual list:
metro_street_running =['141012', '13805', '5397', '13803',
 '13804', '5396', '13802', '5395', '5410', '5411', '13817',
 '12304', '5408', '3821', '2603', '3153', '3124', '378', '65300039',
 '65300038', '15820', '13460', '4994', '1813', '2378', '5049',
 '4652', '2377', '4675', '5040', '65300042', '3674', '15713',
 '13561', '5378', '13560', '70500012', '5377', '15612',
 '12416', '11917', '12415', '8704']

In [19]:
metro_brt_stops = metro_brt_stops >> filter(-_.stop_id.isin(metro_street_running))

In [21]:
# map_hqta(metro_brt_stops)
## OK June 23

In [22]:
act_brt = (rt_utils.get_routes(4, analysis_date)
         >> filter(_.route_id == '1T')
         )

In [23]:
act_brt_stops = routes_to_stops(act_brt, analysis_date)

In [25]:
# map_hqta(act_brt_stops)

In [26]:
# omni_brt = (rt_utils.get_routes(232, analysis_date)
#          >> filter(_.route_short_name == 'sbX')
#          )
## too infrequent!

In [27]:
# omni_brt_stops = routes_to_stops(omni_brt, analysis_date)

In [28]:
# map_hqta(omni_brt_stops)

In [29]:
muni_routes = rt_utils.get_routes(282, analysis_date)

In [30]:
van_ness_brt_route = muni_routes >> filter(_.route_short_name == '49')

In [31]:
van_ness_ids = ['8096', '8097', '18095', '18098', '8094', '8099', '18093', '8100',
'18092', '18101', '18102', '18091', '18103', '8090', '18104', '18089',
'18105', '18088']

In [32]:
van_ness_stops = routes_to_stops(van_ness_brt_route, analysis_date)
van_ness_stops = van_ness_stops >> filter(_.stop_id.isin(van_ness_ids))

In [34]:
# map_hqta(van_ness_stops)

### Ferry

In [35]:
ferry = get_routes_by_type(['4'], analysis_date)

In [36]:
ferry_stops = routes_to_stops(ferry, analysis_date)
angel_and_alcatraz = ['2483552', '2483550', '43002'] ##only stops without bus service, TODO implement algorithm
ferry_stops = ferry_stops >> filter(-_.stop_id.isin(angel_and_alcatraz))

## Combined

In [37]:
rail_brt_ferry = pd.concat([rail_stops, metro_brt_stops, act_brt_stops, van_ness_stops, ferry_stops])

In [38]:
rail_brt_ferry

,calitp_itp_id,stop_id,stop_lat,stop_lon,stop_name,route_type,geometry
74,13,GVB,35.121260,-120.629266,Grover Beach Amtrak,2,POINT (-57308.226 -321533.615)
705,13,SLO,35.276434,-120.654701,San Luis Obispo,2,POINT (-59505.601 -304286.520)
479,13,BFD,35.372140,-119.008210,Bakersfield,2,POINT (90031.416 -293392.047)
889,13,WAC,35.594092,-119.332309,Wasco,2,POINT (60437.770 -268997.529)
807,13,PRB,35.622683,-120.687860,Paso Robles Amtrak Station,2,POINT (-62240.345 -265808.632)
...,...,...,...,...,...,...,...
14,127,43008,37.796533,-122.393763,San Francisco Ferry Building Gate B,4,POINT (-210476.942 -21780.838)
15,127,43003,37.856166,-122.477546,Sausalito Ferry Landing,4,POINT (-217668.884 -14964.633)
17,127,43007,37.872732,-122.455917,Tiburon Ferry Landing,4,POINT (-215721.361 -13173.100)
13,280,890004,37.909510,-122.359250,Richmond Ferry Terminal,4,POINT (-207130.442 -9301.491)


In [39]:
# map_hqta(rail_brt_ferry)

In [40]:
shared_utils.utils.geoparquet_gcs_export(rail_brt_ferry, f'{GCS_FILE_PATH}', f'rail_brt_ferry')

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
